# 0. Env

In [ ]:
import os
import sys
import argparse
from tqdm import tqdm

import torch

from transformers import (
    T5TokenizerFast,
    T5ForConditionalGeneration,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    GenerationConfig,
)

# 1. NMT
- 학습코드: https://github.com/with-rl/nlp-practice/tree/main/src/transformer
- 학습에 사용된 데이터: AI-hub의 모든 번역 데이터 & 기타

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# 사전 학습된 한국어-영어 번역모델
model_name = "cchyun/nmt-koen-t5-small"

In [ ]:
# 모델 및 tokenizer 로딩
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# model이 GPU를 사용하도록
model.to(device)

In [ ]:
# 번역문을 생성할 설정 값
generation_config = GenerationConfig(
    max_new_tokens=128,
    early_stopping=True,
    do_sample=False,
    num_beams=8,
    use_cache=True,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    repetition_penalty=1.2,
    length_penalty=1.0,
)

In [ ]:
# 번역할 원문
line = "만나서 반갑습니다. 저는 홍길동 입니다."

In [ ]:
# 모델에 입력할 token id
x = tokenizer(
        line,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )["input_ids"].to(device)
x

In [ ]:
# 번역문 token id
output = model.generate(
    input_ids=x,
    generation_config=generation_config,
)
output

In [ ]:
# 번역문을 문자로 변환
result = tokenizer.decode(output[0], skip_special_tokens=True)
result

In [ ]:
while True:
    print("input> ", end="")
    line = str(input())
    if len(line) == 0:
        break

    x = tokenizer(
        line,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )["input_ids"].to(device)

    output = model.generate(
        input_ids=x,
        generation_config=generation_config,
    )
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"- ko: {line}\n- en: {result}\n")